In [ ]:
import csv

with open('data.csv', mode='w') as image:
    for i in range(0, 5000):
        
        writer = csv.writer(image, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        if (i >= 0 and i <= 9):
            writer.writerow(["0000" + f'{i}' + '.png'])        
        if (i >= 10 and i <= 99):
            writer.writerow(["000" + f'{i}' + '.png'])
        if (i >= 100 and i <= 999):
            writer.writerow(["00" + f'{i}' + '.png'])
        if (i >= 1000 and i <= 9999):
            writer.writerow(["0" + f'{i}' + '.png'])

In [16]:
from CustomDataset import Custom
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5, 0.5)
])

dir = "D:\\Coding\\Gen-team-test-model\\Mydata"
csv = 'D:\Coding\Gen-team-test-model\data.csv'
dataset = Custom(csv_file= csv, root= dir, transforms= transform)

In [17]:
import torch
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=True)


In [20]:
len(dataset)

5000

In [10]:
dataset[5]

tensor([[[-0.8196, -0.8275, -0.8588,  ...,  0.6314,  0.6000,  0.5765],
         [-0.8745, -0.8824, -0.9059,  ...,  0.6235,  0.5922,  0.5765],
         [-0.9137, -0.9294, -0.9451,  ...,  0.6471,  0.5843,  0.5765],
         ...,
         [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  0.9922,  0.9922],
         [ 0.9922,  0.9922,  0.9922,  ...,  1.0000,  0.9922,  1.0000],
         [ 0.9922,  0.9922,  0.9843,  ...,  1.0000,  1.0000,  1.0000]],

        [[-0.8118, -0.8196, -0.8510,  ...,  0.5451,  0.5216,  0.5059],
         [-0.8667, -0.8745, -0.8980,  ...,  0.5451,  0.5216,  0.5137],
         [-0.9059, -0.9216, -0.9373,  ...,  0.5765,  0.5137,  0.5137],
         ...,
         [-0.0824, -0.0824, -0.0902,  ..., -0.0118, -0.0196, -0.0196],
         [-0.0902, -0.0902, -0.0980,  ..., -0.0275, -0.0275, -0.0118],
         [-0.1059, -0.1059, -0.1059,  ..., -0.0196, -0.0118, -0.0118]],

        [[-0.8431, -0.8510, -0.8902,  ...,  0.4275,  0.3961,  0.3725],
         [-0.9059, -0.9137, -0.9373,  ...,  0

In [ ]:
import pandas as pd
pd.read_csv('data.csv')